In [2]:
from purreaddocs import *

In [3]:
import os
import sys
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, AIMessage, HumanMessage

myopenaikey = readopenaikey()
chat = ChatOpenAI( temperature = 0.3, max_tokens = 150, openai_api_key = myopenaikey )

chat (
    [  
        SystemMessage( content = "Hello, I am a chatbot. I am very friendly." ) , 
        AIMessage( content = "Hello, I am a chatbot. I am very friendly." ), 
        HumanMessage( content = "Hello, I am a human. I am very friendly." )
    ]
)

AIMessage(content="Hello, nice to meet you! It's great to hear that you're friendly. How can I assist you today?", additional_kwargs={}, example=False)

# LLM output

In [4]:
from langchain.llms import OpenAI

llm = OpenAI ( model_name = "text-davinci-003", openai_api_key = myopenaikey)

llm(
    "What is the name of President of USA"
)

'\n\nThe current President of the United States is Joe Biden.'

In [5]:
a = input()
print (a)

a


### Generate Vector Embeddings

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embds = OpenAIEmbeddings( openai_api_key = myopenaikey )

text = "I am a human. I am very friendly."

txtembeds = embds.embed_query( text )

print(len(txtembeds))
print (txtembeds)

1536
[-0.01396973803639412, -0.003000765573233366, -0.016773836687207222, -0.009078426286578178, -0.00943369697779417, 0.009135523810982704, -0.001000783871859312, 0.016037918627262115, -0.02085944451391697, -0.010099828243255615, -0.007682721596211195, -0.009338535368442535, 0.002588398288935423, -0.013474896550178528, -0.015517701394855976, -0.02451365441083908, 0.03024873323738575, -0.019654063507914543, 0.008399605751037598, -0.003850876819342375, -0.008799285627901554, 0.005760455038398504, 0.002466274192556739, -0.019311482086777687, -0.021214716136455536, 0.023295585066080093, 0.024716665968298912, -0.009611331857740879, 0.013005432672798634, -0.0323803536593914, 0.020948262885212898, -0.009008641354739666, 0.0025947424583137035, -0.009465416893362999, 0.0007212464115582407, -0.013703284785151482, 0.0008707295637577772, -0.004117329604923725, -0.004558245651423931, -0.01712910644710064, 0.004704160150140524, 0.00015533165424130857, -0.0018540672026574612, -0.010302839800715446, 

### Prompt Templates

In [7]:
from langchain import PromptTemplate


myprompt = """
What is the name of President of {country} ?
"""

pt1 = PromptTemplate( 
    input_variables = ["country"], 
    template = myprompt)


final_prompt = pt1.format( country = "UK" )

print (f" LLM return value : {llm( final_prompt )} " )

 LLM return value : 
The United Kingdom does not have a president. The head of state of the UK is Queen Elizabeth II. 


# Output Parser

In [8]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

contents = readSampleDoc()



response_schemas1 =  [
    ResponseSchema(name = "ssn", description = "value of ssn"),
    ResponseSchema(name = "US Passport", description = "value of US Passport"),
    ResponseSchema(name = "India Passport", description = "value of India Passport"),
    ResponseSchema(name = "Credit Card", description = "value of Credit Card"),
    ResponseSchema(name = "License", description = "value of License"),
]

output_parsers = StructuredOutputParser.from_response_schemas( response_schemas1 ) 
format_instructions = output_parsers.get_format_instructions()
#print (format_instructions)

myprompt = f"""
Identify Sensitive data in the text and return name value in JSON format ?

{format_instructions}

Text : {contents}
"""
llm_output = llm( myprompt )
print (llm_output)

output_json = output_parsers.parse( llm_output )
print(type(output_json))
#output_json1 = eval(llm_output)
print (output_json)




```json
{
	"ssn": "N/A",
	"US Passport": "N/A",
	"India Passport": "N/A",
	"Credit Card": "1234-5678-9012-3456",
	"License": "A1234567"
}
```
<class 'dict'>
{'ssn': 'N/A', 'US Passport': 'N/A', 'India Passport': 'N/A', 'Credit Card': '1234-5678-9012-3456', 'License': 'A1234567'}


# Chat History

In [9]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_ai_message( "" )
history.add_user_message( "Hello, what is the population of California." )
#history.add_system_message( "provide  numbers in 2023" )

ai_response = chat (history.messages)
print (ai_response)

history.add_ai_message( ai_response.content )  

print (history.messages)

content='As of 2021, the estimated population of California is approximately 39.5 million people.' additional_kwargs={} example=False
[AIMessage(content='', additional_kwargs={}, example=False), HumanMessage(content='Hello, what is the population of California.', additional_kwargs={}, example=False), AIMessage(content='As of 2021, the estimated population of California is approximately 39.5 million people.', additional_kwargs={}, example=False)]


In [10]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

myprompt1 = """
What is the capital of this coutry?
% CAPITAL
{country} ?
"""

my_prompt_template1 = PromptTemplate( 
    input_variables = ["country"], 
    template = myprompt1)

pname_chian = LLMChain (llm = llm, prompt = my_prompt_template1)


myprompt2 = """
whats is the population of this city
% CITY
{country}
"""

my_prompt_template2 = PromptTemplate( 
    input_variables = ["country"], 
    template = myprompt2)   

wname_chain = LLMChain (llm = llm, prompt = my_prompt_template2)

complete_chain = SimpleSequentialChain( chains = [pname_chian, wname_chain], verbose = True )  

llm_out1 = complete_chain.run ("USA")  

print (llm_out1)



> Entering new  chain...
Washington, D.C.

As of July 2019, the population of Washington, D.C. was 702,455.

> Finished chain.

As of July 2019, the population of Washington, D.C. was 702,455.


# ReEx pattern for Classification

In [12]:
from langchain.llms import AzureOpenAI

contents = readSampleDoc()

myprompt1 = f"""
identify regex pattern ^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$ in the text and return name value in JSON format with name as mycustclass ?

{contents}
"""

llm_out1 = llm( myprompt1 )
print (llm_out1)

{ "name": "mycustclass", "value": "roshnip@microsoft.com" }
